[ResNet of roboflow](https://colab.research.google.com/github/captcha-challenger/hcaptcha-model-factory/blob/main/automation/roboflow_resnet.ipynb)

In [ ]:
!nvidia-smi

In [ ]:
!pip install loguru onnx fire hcaptcha-challenger[sentinel]
!git clone https://github.com/captcha-challenger/hcaptcha-model-factory.git
!mv -f hcaptcha-model-factory/src .
!rm -rf hcaptcha-model-factory/

Set `label_name` and `GITHUB_TOKEN`

In [ ]:
import os

import hcaptcha_challenger as solver

os.environ["GITHUB_TOKEN"] = ""
task_name = "robot"
onnx_archive_name = "robot2312"

solver.diagnose_task(task_name)

Upload datasets to `[PROJECT_DIR]/database2309/`

In [ ]:
from pathlib import Path
from loguru import logger
import zipfile

this_dir = Path("/content")
project_dir = this_dir

model_dir = project_dir.joinpath("model")
factory_data_dir = project_dir.joinpath("data")
source_dir = project_dir.joinpath("src")
zip_path = project_dir.joinpath(f"{task_name}.zip")

if not zip_path.exists():
    raise FileNotFoundError
with zipfile.ZipFile(zip_path) as z:
    z.extractall(factory_data_dir.joinpath(task_name))

Training

In [ ]:
%cd {source_dir}

from factories.resnet import ResNet


# - INPUT: `[PROJECT]/data/<task_name>`
# - OUTPUT: `[PROJECT]/model/<task_name>/<task_name.onnx>`

model = ResNet(
    task_name=task_name,
    epochs=None,  # default to 200
    batch_size=None,  # default to 4
    dir_dataset=str(factory_data_dir),
    dir_model=str(model_dir),
)
model.train()
model.conv_pth2onnx(verbose=False)

Deployment model to GitHub

In [ ]:
import locale

locale.getpreferredencoding = lambda d=True: "UTF-8"

!pip install PyGithub

import shutil

from github import Auth, Github, GithubException

def quick_development():
    if not os.getenv("GITHUB_TOKEN"):
        logger.warning("Skip model deployment, miss GITHUB TOKEN")
        return

    auth = Auth.Token(os.getenv("GITHUB_TOKEN", ""))
    repo = Github(auth=auth).get_repo("QIN2DIM/hcaptcha-challenger")
    modelhub_title = "ONNX ModelHub"

    model_path = model_dir.joinpath(task_name, f"{task_name}.onnx")
    pending_onnx_path = model_dir.joinpath(task_name, f"{onnx_archive_name}.onnx")
    shutil.copy(model_path, pending_onnx_path)

    for release in repo.get_releases():
        if release.title != modelhub_title:
            continue
        try:
            res = release.upload_asset(path=str(pending_onnx_path))
        except GithubException as err:
            if err.status == 422:
                logger.error(
                    f"The model file already exists, please manually replace the file with the same name - url={repo.releases_url}",
                    url=repo.releases_url,
                )
        except Exception as err:
            logger.error(err)
        else:
            logger.success(
                f"Model file uploaded successfully - name={res.name} url={res.browser_download_url}"
            )


quick_development()